# Travail préparatoire

Avant de commencer le TP, nous vous recommandons fortement de :
- Suivre le tutoriel [Customize what happens in Model.fit](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit)
- Vous reporter si besoin à la documentation de la classe [``` Model ```](https://www.tensorflow.org/api_docs/python/tf/keras/Model)

# Objectifs 

Les objectifs de ce TP sont : 
- Découvrir le principe de la compression des réseaux de neurones par distillation (cf: Hinton et al., [Distilling the Knowledge in a Neural Network](https://arxiv.org/abs/1503.02531))

- Illustrer la flexibilité du paradigme des réseaux de neurones profonds

- Apprendre à utiliser la classe ``` Model ``` et à redéfinir certaines de ses méthodes (cf. travail préparatoire)





# Travail à réaliser 

Lors de ce TP, nous allons réaliser la distillation d'un réseau de neurones de type CNN (le *Teacher*) dans un réseau plus léger (le "Student"). Pour ce cas d'étude nous utiliserons la base MNIST que nous avons déjà utilisé lors des TP précédents. 
Vous allez donc devoir : 
- Définir l'architecture du réseau *Teacher* et optimiser le modèle sur la base MNIST (à noter que vous pouvez également utiliser un modèle pré-appris pour la tâche qui nous intéresse, ici la reconnaissance de chiffre manuscript)
- Définir l'architecture du réseau léger *Student*
- Préparer les données d'apprentissage qui serviront à la distillation. Nous utiliserons les données de MNIST (les mêmes que celles qui ont servi à l'apprentissage du Teacher, mais ce n'est pas une obligation, les deux bases peuvent être différentes, seules les tâches à réaliser doivent être identiques)
- Implémenter la classe *Distiller* qui sera en charge de la distillation. 

La classe ``` Distiller ``` héritera de la classe ``` Model ``` pour laquelle il faudra redéfinir le constructeur, et les méthodes ``` train_step ``` et ``` test_step ```. Vous pourrez également redéfinir la méthode ``` compile ``` si vous souhaitez faire un code plus générique et tester différentes fonctions de coût et hyper-paramètre propre à la méthode de distillation.





In [1]:
import tensorflow as tf
from tensorflow.keras import Model

## Préparation des données 

In [2]:
## Chargement et normalisation des données
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0

# POUR LES CNN : On rajoute une dimension pour spécifier qu'il s'agit d'images en NdG
train_images = train_images.reshape(-1,28,28,1)
test_images = test_images.reshape(-1,28,28,1)

# One hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

In [9]:
train_images.shape

(60000, 28, 28, 1)

## Définition et apprentissage de modèle ```teacher```


Définition du modèle

In [4]:
## DEFINITION DES MODELES
## Teacher 
## Définition de l'architecture du modèle
 
# 16@3x3 -> AvPool -> 32@3x3 -> AvPool -> 64@3x3 -> AvPool -> FC 1024 -> FC 512
teacher = tf.keras.models.Sequential()

teacher.add(tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),padding="same", activation='tanh', input_shape=(28, 28, 1)))
teacher.add(tf.keras.layers.AveragePooling2D())

teacher.add(tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),padding="valid", activation='tanh'))
teacher.add(tf.keras.layers.AveragePooling2D())

teacher.add(tf.keras.layers.Flatten())

teacher.add(tf.keras.layers.Dense(1024 , activation='tanh'))
teacher.add(tf.keras.layers.Dense(512 , activation='tanh'))

teacher.add(tf.keras.layers.Dense(10 , activation='softmax'))



In [5]:
print(teacher.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
average_pooling2d_1 (Average (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 64)        9280      
_________________________________________________________________
average_pooling2d_2 (Average (None, 6, 6, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2360320   
_________________________________________________________________
dense_1 (Dense)              (None, 512)              

Apprentissage du modèle (Adam + Entropie Croisée sur 10 epochs)


In [23]:
load_teacher = False

In [44]:
# Pour éviter d'apprendre à chaque fois le réseau Teacher, on l'enregistrer et 
# on le recharche si besoin

 
if (load_teacher == True):
    
    # chargement du Teacher
    teacher = keras.models.load_model('./teacher.h5')
    
else:
    # Apprentissage du modèle Teacher
    sgd = tf.keras.optimizers.Adam()
    teacher.compile(sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    teacher.fit(train_images,
         train_labels,
         batch_size=64,
         epochs=20
         )
    
    # Enregistrement du modele
    teacher.save('./teacher.h5')
    load_teacher == True
    

Epoch 1/20
938/938 [==============================] - 53s 56ms/step - loss: 0.0075 - accuracy: 0.9974
Epoch 2/20
938/938 [==============================] - 54s 57ms/step - loss: 0.0105 - accuracy: 0.9966
Epoch 3/20
938/938 [==============================] - 55s 59ms/step - loss: 0.0111 - accuracy: 0.9961
Epoch 4/20
938/938 [==============================] - 56s 60ms/step - loss: 0.0093 - accuracy: 0.9970
Epoch 5/20
938/938 [==============================] - 56s 60ms/step - loss: 0.0099 - accuracy: 0.99660s - loss: 0.0099 - accuracy: 0.
Epoch 6/20
938/938 [==============================] - 56s 59ms/step - loss: 0.0118 - accuracy: 0.9964
Epoch 7/20
938/938 [==============================] - 56s 60ms/step - loss: 0.0087 - accuracy: 0.9972
Epoch 8/20
938/938 [==============================] - 54s 58ms/step - loss: 0.0098 - accuracy: 0.9967
Epoch 9/20
938/938 [==============================] - 54s 57ms/step - loss: 0.0081 - accuracy: 0.9974
Epoch 10/20
938/938 [=============================

Evaluation des performances sur la base de test

In [45]:
test_loss, test_acc = teacher.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

313/313 [==============================] - 2s 8ms/step - loss: 0.0824 - accuracy: 0.9846
Test accuracy: 0.9846000075340271


## Définition du modèle  ```student```


In [30]:
## Student
student  = tf.keras.models.Sequential()

student.add(tf.keras.layers.Conv2D(filters=8,kernel_size=(3,3),padding="same", activation='tanh', input_shape=(28, 28, 1)))
student.add(tf.keras.layers.AveragePooling2D())

student.add(tf.keras.layers.Conv2D(filters=8,kernel_size=(3,3),padding="valid", activation='tanh'))
student.add(tf.keras.layers.AveragePooling2D())

student.add(tf.keras.layers.Flatten())

student.add(tf.keras.layers.Dense(64 , activation='tanh'))
student.add(tf.keras.layers.Dense(32 , activation='tanh'))

student.add(tf.keras.layers.Dense(10 , activation='softmax'))


print(student.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 28, 28, 8)         80        
_________________________________________________________________
average_pooling2d_8 (Average (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 12, 12, 8)         584       
_________________________________________________________________
average_pooling2d_9 (Average (None, 6, 6, 8)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 288)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                18496     
_________________________________________________________________
dense_7 (Dense)              (None, 32)               

In [14]:
# On copie l'instance pour comparer les différentes stratégies d'apprentissage 
student_loss_sup =  tf.keras.models.clone_model(student)
student_loss_distillation =  tf.keras.models.clone_model(student)
student_loss_both =  tf.keras.models.clone_model(student)

## Définition de la classe ``` Distiller ```
 
Le distiller a besoin du modèle ``` teacher ``` appris et de modèle ``` student ``` . Il a également besoin de parametres specifique à la distilation (eg. coeff de pondération des fonctions de coût) 
 
Les méthodes ``` train_step ``` et ``` test_step ``` doit être redéfinies et seront appelées respectivement par les méthodes ``` fit ``` et  ``` evaluate ```

In [37]:
class Distiller(Model):
 
    def __init__(self, teacher, student, coef):
        super(Distiller, self).__init__()

        self.teacher = teacher
        self.student = student
        self.coef = coef
        
    def train_step(self, data):
        
        x,y = data
        
        y_teacher = self.teacher(x, training=False)
        
        with tf.GradientTape() as tape:
            y_pred = self.student(x, training=True)
            loss_sup = self.compiled_loss(y, y_pred, regularization_losses=self.losses)
            loss_dist = self.compiled_loss(y_teacher, y_pred, regularization_losses=self.losses)
            a = self.coef
            loss = a * loss_dist + (1-a) *loss_sup
            
        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        x, y = data
        y_pred = self.student(x, training=False)
        
        self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        # Update the metrics.
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}
        

## Distillation du modèle 

Apprentissage du modèle léger

In [38]:
# Uniquement la loss superviée
distiller = Distiller(teacher, student, 0)
sgd = tf.keras.optimizers.Adam()
distiller.compile(sgd, loss='categorical_crossentropy', metrics=['accuracy'])

distiller.fit(train_images,
         train_labels,
         batch_size=120,
         epochs=20
         )

Epoch 1/20
500/500 [==============================] - 18s 35ms/step - loss: 0.4720 - accuracy: 0.8671
Epoch 2/20
500/500 [==============================] - 22s 44ms/step - loss: 0.1587 - accuracy: 0.9538
Epoch 3/20
500/500 [==============================] - 22s 45ms/step - loss: 0.1044 - accuracy: 0.9694
Epoch 4/20
500/500 [==============================] - 20s 39ms/step - loss: 0.0798 - accuracy: 0.9770
Epoch 5/20
500/500 [==============================] - 20s 39ms/step - loss: 0.0665 - accuracy: 0.9806
Epoch 6/20
500/500 [==============================] - 20s 40ms/step - loss: 0.0561 - accuracy: 0.9839
Epoch 7/20
500/500 [==============================] - 20s 40ms/step - loss: 0.0488 - accuracy: 0.9859
Epoch 8/20
500/500 [==============================] - 20s 40ms/step - loss: 0.0427 - accuracy: 0.9880
Epoch 9/20
500/500 [==============================] - 21s 41ms/step - loss: 0.0391 - accuracy: 0.98900s - loss: 0.0392 - accuracy: 
Epoch 10/20
500/500 [==============================]

Evaluation du modèle

In [39]:
test_loss, test_acc = distiller.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

313/313 [==============================] - 1s 3ms/step - loss: 0.0507 - accuracy: 0.9848
Test accuracy: 0.9847999811172485


In [40]:
# Uniquement la loss distillation

distiller = Distiller(teacher, student, 1)
sgd = tf.keras.optimizers.Adam()
distiller.compile(sgd, loss='categorical_crossentropy', metrics=['accuracy'])

distiller.fit(train_images,
         train_labels,
         batch_size=120,
         epochs=20
         )

Epoch 1/20
500/500 [==============================] - 18s 37ms/step - loss: 0.0182 - accuracy: 0.9963
Epoch 2/20
500/500 [==============================] - 20s 41ms/step - loss: 0.0169 - accuracy: 0.9961
Epoch 3/20
500/500 [==============================] - 19s 39ms/step - loss: 0.0155 - accuracy: 0.9966
Epoch 4/20
500/500 [==============================] - 20s 40ms/step - loss: 0.0150 - accuracy: 0.9966
Epoch 5/20
500/500 [==============================] - 20s 39ms/step - loss: 0.0143 - accuracy: 0.9966
Epoch 6/20
500/500 [==============================] - 20s 40ms/step - loss: 0.0128 - accuracy: 0.9973
Epoch 7/20
500/500 [==============================] - 20s 40ms/step - loss: 0.0134 - accuracy: 0.9969
Epoch 8/20
500/500 [==============================] - 21s 41ms/step - loss: 0.0130 - accuracy: 0.9968
Epoch 9/20
500/500 [==============================] - 21s 41ms/step - loss: 0.0124 - accuracy: 0.9970
Epoch 10/20
500/500 [==============================] - 21s 42ms/step - loss: 0.012

In [41]:
test_loss, test_acc = distiller.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

313/313 [==============================] - 1s 3ms/step - loss: 0.0503 - accuracy: 0.9862
Test accuracy: 0.9861999750137329


In [42]:
# les 2 loss
distiller = Distiller(teacher, student, 0.5)
sgd = tf.keras.optimizers.Adam()
distiller.compile(sgd, loss='categorical_crossentropy', metrics=['accuracy'])

distiller.fit(train_images,
         train_labels,
         batch_size=120,
         epochs=20
         )

Epoch 1/20
500/500 [==============================] - 18s 36ms/step - loss: 0.0098 - accuracy: 0.9976
Epoch 2/20
500/500 [==============================] - 20s 40ms/step - loss: 0.0082 - accuracy: 0.9986
Epoch 3/20
500/500 [==============================] - 20s 40ms/step - loss: 0.0088 - accuracy: 0.9984
Epoch 4/20
500/500 [==============================] - 20s 39ms/step - loss: 0.0081 - accuracy: 0.9987
Epoch 5/20
500/500 [==============================] - 21s 41ms/step - loss: 0.0078 - accuracy: 0.9988
Epoch 6/20
500/500 [==============================] - 20s 40ms/step - loss: 0.0097 - accuracy: 0.9979
Epoch 7/20
500/500 [==============================] - 20s 40ms/step - loss: 0.0074 - accuracy: 0.9989
Epoch 8/20
500/500 [==============================] - 20s 40ms/step - loss: 0.0075 - accuracy: 0.9990
Epoch 9/20
500/500 [==============================] - 20s 41ms/step - loss: 0.0070 - accuracy: 0.9992
Epoch 10/20
500/500 [==============================] - 20s 41ms/step - loss: 0.009

In [43]:
test_loss, test_acc = distiller.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

313/313 [==============================] - 1s 3ms/step - loss: 0.0581 - accuracy: 0.9853
Test accuracy: 0.9853000044822693


Dans ce TP, nous avons implémenté et évaluer une stratégie de distillation de l'information d'un réseau Teacher (expert) vers un réseau Student La distillation peut-être utilisée pour :
- compresser la taille (nombre de paramètre) d'un réseau expert
- spécialiser un réseau léger pour un domaine particulier
- apprendre un réseau lorsque l'on dispose d'un (ou plusieurs) réseau mais pas de données annotées. 


Vous pouvez également tester cette stratégie sur : 
- d'autres bases (e.g. CIFAR 10)
- en utilisant des réseaux pré-apris disponibles dans TF2 (eg: https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1) -> cf: https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub pour un exemple d'utilisation de modèles pré-appris